In [2]:
#!pip install pycaret[full]
#!pip install explainerdashboard
#!pip install --upgrade fairlearn==0.7.0 raiwidgets
#!pip install scikit-learn==0.23.2

# Data Import

## Import Modules

In [13]:
import os
from pathlib import Path
from re import search
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.classification import setup, compare_models, create_model, tune_model, evaluate_model, load_model, plot_model, add_metric, dashboard, check_fairness, create_api, create_docker, create_app
from utils import  adapt_datatype

## Set Paths

In [4]:
# set paths
ROOTDIR = os.getcwd()
DATAPATH = os.path.join(ROOTDIR, "data")
MODELPATH = os.path.join(ROOTDIR, "model")
Path(DATAPATH).mkdir(parents=True, exist_ok=True)
Path(MODELPATH).mkdir(parents=True, exist_ok=True)

## Import Customer Churn Dataset

In [5]:
# import data
filename = 'telco_customer_churn.csv'
df = pd.read_csv(os.path.join(DATAPATH, filename))
# adapt data types
df = adapt_datatype(df)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Setup the Auto-ML Experiment

In [6]:
# pycaret does all the data prep
setup(df, 
    target='Churn', 
    ignore_features=['customerID'],
    data_split_stratify=True,
    train_size=0.6,
    fold=5,
    log_experiment=True,
    experiment_name='churn_prediction',
    session_id = 123,
    silent=True)

,Description,Value
0,session_id,123
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,"No: 0, Yes: 1"
4,Original Data,"(7043, 21)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,16
8,Ordinal Features,False
9,High Cardinality Features,False


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False,
                                       features_todrop=['customerID'],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                 ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                 ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                 ('cluster_all', 'passthrough'),
                 ('dummy', Dummify

# Load Trained Model Pipeline

In [7]:
# load model
pipeline = load_model(os.path.join(MODELPATH, 'churn_model_pipeline'))
pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False,
                                      features_todrop=['customerID'],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Churn',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                n...
                ('dummy', Dummify(target='Churn')),
                ('fix_perfect', Remove_100(target='Churn')),
                ('clean_names', Clean_Colum_Names()),
                ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
 

# Create Web API using Fast-API

In [8]:
model = pipeline['trained_model']
model

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage='auto',
                           solver='lsqr', store_covariance=False, tol=0.0001)

In [9]:
help(create_api)

Help on function create_api in module pycaret.classification:

create_api(estimator, api_name: str, host: str = '127.0.0.1', port: int = 8000) -> None
    This function takes an input ``estimator`` and creates a POST API for
    inference. It only creates the API and doesn't run it automatically.
    To run the API, you must run the Python file using ``!python``.
    
    
    Example
    -------
    >>> from pycaret.datasets import get_data
    >>> juice = get_data('juice')
    >>> from pycaret.classification import *
    >>> exp_name = setup(data = juice,  target = 'Purchase')
    >>> lr = create_model('lr')
    >>> create_api(lr, 'lr_api'
    >>> !python lr_api.py
    
    
    estimator: scikit-learn compatible object
        Trained model object
    
    
    api_name: scikit-learn compatible object
        Trained model object
    
    
    host: str, default = '127.0.0.1'
        API host address.
    
    
    port: int, default = 8000
        port for API.
    
    
    Return

In [10]:
# create fast-api
create_api(model, 'churn_api', port=5000)


API sucessfully created. This function only creates a POST API, it doesn't run it automatically.

To run your API, please run this command --> !python churn_api.py
    


# Create Docker File and Requirements.txt for Productionizing the Fast_API

In [11]:
help(create_docker)

Help on function create_docker in module pycaret.classification:

create_docker(api_name: str, base_image: str = 'python:3.8-slim', expose_port: int = 8000) -> None
    This function creates a ``Dockerfile`` and ``requirements.txt`` for
    productionalizing API end-point.
    
    
    Example
    -------
    >>> from pycaret.datasets import get_data
    >>> juice = get_data('juice')
    >>> from pycaret.classification import *
    >>> exp_name = setup(data = juice,  target = 'Purchase')
    >>> lr = create_model('lr')
    >>> create_api(lr, 'lr_api')
    >>> create_docker('lr_api')
    
    
    api_name: str
        Name of API. Must be saved as a .py file in the same folder.
    
    
    base_image: str, default = "python:3.8-slim"
        Name of the base image for Dockerfile.
    
    
    expose_port: int, default = 8000
        port for expose for API in the Dockerfile.
    
    
    Returns:
        None



In [12]:
# create docker file
create_docker('churn_api')

Writing requirements.txt
Writing Dockerfile
Dockerfile and requirements.txt successfully created.
To build image you have to run --> !docker image build -f "Dockerfile" -t IMAGE_NAME:IMAGE_TAG .
        
